In [1]:
!pip install cmake 'gym[atari]' scipy

     |████████████████████████████████| 17.7 MB 4.4 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 75.3 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 56.0 MB/s eta 0:00:01
     |████████████████████████████████| 2.8 MB 62.5 MB/s eta 0:00:01
     |████████████████████████████████| 49.5 MB 221 kB/s  eta 0:00:01     |████████████████████████████    | 43.5 MB 31.8 MB/s eta 0:00:01
     |████████████████████████████████| 829 kB 57.9 MB/s eta 0:00:01
  Created wheel for gym: filename=gym-0.17.3-py3-none-any.whl size=1654654 sha256=0ea08184f7b83382f248fbbd8822fff035f1bc2a01bf9b83341dfb34f5647311
  Stored in directory: /home/jovyan/.cache/pip/wheels/95/b0/62/af38051b97354eab5b2ff9a5fb92f5015a480745568c91e0b0
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491058 sha256=038682a47d4fd55fe9c44285644bb6fcd78510a2ec51d8a9bf8b4583171ee01c
  Stored in directory: /home/jovyan/.cache/pip/wheels/6e/9c/ed/4499c9865ac1002697793e0ae05ba6be33553d09

In [2]:
import gym
env = gym.make("Taxi-v3").env
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [4]:
env.reset()
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [8]:
print("Action Space: {}".format(env.action_space))

Action Space: Discrete(6)


In [9]:
print("Action Space: {}".format(env.observation_space))

Action Space: Discrete(500)


In [13]:
state = env.encode(3,1,2,0) #row taxi, col taxi, psnger index, dest index)
print("State: ", state)
env.s = state
env.render()

State:  328
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [14]:
env.P[328] #returns-   {action: [(prob, nextstate, reward, done)]}

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

In [20]:
env.s = 328

epochs = 0
penalties, reward = 0, 0

frames = []
done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1
    
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    
    
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 1233
Penalties incurred: 406
epochs? 1233


In [24]:
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [25]:
import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode:{i}")
        
print("training finished. \n")

Episode:100000
training finished. 



In [28]:
q_table[328]

array([ -2.39919844,  -2.27325184,  -2.40556451,  -2.35320673,
       -10.5975519 , -11.00166454])

In [29]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties = 0, 0
episodes = 100

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 100 episodes:
Average timesteps per episode: 12.85
Average penalties per episode: 0.0


In [ ]:
#https://www.learndatasci.com/tutorials/reinforcement-q-learning-scratch-python-openai-gym/